In [1]:
import random
import pickle

import numpy as np
import pandas as pd
from nltk.tokenize import RegexpTokenizer

from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import LSTM, Dense, Activation
from tensorflow.keras.optimizers import RMSprop

In [2]:
text_df = pd.read_csv("fake_or_real_news.csv")

In [3]:
text = list(text_df.text.values)
joined_text = " ".join(text)

In [4]:
partial_text = joined_text[:10000]

In [5]:
tokenizer = RegexpTokenizer(r"\w+")
tokens = tokenizer.tokenize(partial_text.lower())

In [6]:
unique_tokens = np.unique(tokens)
unique_token_index = {token: idx for idx, token in enumerate(unique_tokens)}

In [7]:
n_words = 10
input_words = []
next_words = []

for i in range(len(tokens) - n_words):
  input_words.append(tokens[i:i+n_words])
  next_words.append(tokens[i+n_words])

In [8]:
x = np.zeros((len(input_words), n_words, len(unique_tokens)), dtype=bool)
y = np.zeros((len(next_words), len(unique_tokens)), dtype=bool)

In [9]:
for i,words in enumerate(input_words):
  for j, word in enumerate(words):
    x[i, j, unique_token_index[word]] = 1
  y[i, unique_token_index[next_words[i]]] = 1

In [10]:
model = Sequential()
model.add(LSTM(128, input_shape=(n_words,len(unique_tokens)), return_sequences=True))
model.add(LSTM(128))
model.add(Dense(len(unique_tokens)))
model.add(Activation("softmax"))

In [11]:
model.compile(loss="categorical_crossentropy", optimizer=RMSprop(learning_rate=0.01), metrics=["accuracy"])
model.fit(x,y,batch_size=128, epochs=30, shuffle=True)

Epoch 1/30
14/14 [==============================] - 6s 154ms/step - loss: 6.1782 - accuracy: 0.0395
Epoch 2/30
14/14 [==============================] - 2s 110ms/step - loss: 5.8491 - accuracy: 0.0601
Epoch 3/30
14/14 [==============================] - 1s 88ms/step - loss: 5.8025 - accuracy: 0.0618
Epoch 4/30
14/14 [==============================] - 1s 89ms/step - loss: 5.7671 - accuracy: 0.0618
Epoch 5/30
14/14 [==============================] - 1s 90ms/step - loss: 5.7396 - accuracy: 0.0618
Epoch 6/30
14/14 [==============================] - 1s 88ms/step - loss: 5.6796 - accuracy: 0.0618
Epoch 7/30
14/14 [==============================] - 1s 89ms/step - loss: 5.6321 - accuracy: 0.0595
Epoch 8/30
14/14 [==============================] - 1s 101ms/step - loss: 5.5245 - accuracy: 0.0675
Epoch 9/30
14/14 [==============================] - 2s 103ms/step - loss: 5.3410 - accuracy: 0.0841
Epoch 10/30
14/14 [==============================] - 2s 149ms/step - loss: 5.1791 - accuracy: 0.0835
Epoc

In [12]:
model.save("Textmodel.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [13]:
model = load_model("Textmodel.h5")

In [14]:
def predict_next_word(input_text, n_best):
  input_text = input_text.lower()
  x = np.zeros((1, n_words, len(unique_tokens)))
  for i,word in enumerate(input_text.split()):
    x[0, i, unique_token_index[word]] = 1

  predictions = model.predict(x)[0]
  return np.argpartition(predictions, -n_best)[-n_best:]

In [18]:
possible = predict_next_word("She has to go now and",5)

1/1 [==============================] - 1s 1s/step


In [ ]:
print([unique_tokens[idx] for idx in possible])

In [20]:
def generate_text(input_text, text_length, creativity=3):
  word_sequence = input_text.split()
  current = 0
  for _ in range(text_length):
    sub_sequence = " ".join(tokenizer.tokenize(" ".join(word_sequence).lower())[current : current+n_words])
    try:
      choice = unique_tokens[random.choice(predict_next_word(sub_sequence, creativity))]
    except:
      choice = random.choice(unique_tokens)
    word_sequence.append(choice)
    current += 1
  return " ".join(word_sequence)

In [ ]:
generate_text("She has to go now and", 100, 5)